# Create ResNet

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import * 
#import torch 
#from torch import nn 
#import torchvision

In [3]:
bs = 16

In [4]:
path = %pwd
path = Path(f'{path}/../data/')

In [5]:
data = ImageDataBunch.from_folder(path, train="train_melspecs", valid='val_melspecs', ds_tfms=None, size=128, bs=bs)
data.normalize(imagenet_stats); 

In [6]:
def conv(ni, nf):
    return nn.Conv2d(ni,nf, kernel_size=3, stride=2, padding=1)

In [7]:
model = nn.Sequential(conv(3,8), 
nn.BatchNorm2d(8), 
nn.ReLU(), 
conv(8,16), 
nn.BatchNorm2d(16),
nn.ReLU(), 
conv(16,32),
nn.BatchNorm2d(32),
nn.ReLU(), 
conv(32,16),
nn.BatchNorm2d(16), 
nn.ReLU(), 
conv(16,3),
nn.BatchNorm2d(3), 
Flatten()
)

In [8]:
learn = Learner(data, model, loss_func=nn.CrossEntropyLoss(), metrics=accuracy)

In [9]:
learn.summary()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [8, 64, 64]          224        True      
______________________________________________________________________
BatchNorm2d          [8, 64, 64]          16         True      
______________________________________________________________________
ReLU                 [8, 64, 64]          0          False     
______________________________________________________________________
Conv2d               [16, 32, 32]         1,168      True      
______________________________________________________________________
BatchNorm2d          [16, 32, 32]         32         True      
______________________________________________________________________
ReLU                 [16, 32, 32]         0          False     
______________________________________________________________________
Conv2d               [32, 16, 16]         4,640      True      
___________________________________________________

In [10]:
#learn.lr_find()

In [11]:
#learn.recorder.plot()

In [12]:
#learn.fit_one_cycle(20, slice(1e-2))

## Refaktorering

In [13]:
def conv2(ni,nf): 
    return conv_layer(ni,nf,stride=2)

In [14]:
conv_layer(3,8)

Sequential(
  (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): ReLU(inplace)
  (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [15]:
model = nn.Sequential(conv2(3,8), 
                      conv2(8,16), 
                      conv2(16,32), 
                      conv2(32,16),
                      conv2(16,3), 
                      Flatten()
)

In [16]:
learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

In [17]:
#learn.fit_one_cycle(15, max_lr=1e-2)

## ResNet block

In [18]:
class ResBlock(nn.Module): 
    def __init__(self, nf):
        super().__init__()
        self.conv1 = conv_layer(nf,nf)
        self.conv2 = conv_layer(nf,nf)
    
    def forward(self, x): 
        print(x.shape)
        print(self.conv1(x).shape)
        print(self.conv2(self.conv1(x)).shape)
        print("neste")
        return x + self.conv2(self.conv1(x))

In [19]:
model = nn.Sequential(
    conv2(3,8), 
    res_block(8),
    conv2(8,16),
    res_block(16), 
    conv2(16,32), 
    res_block(32), 
    conv2(32,16), 
    res_block(16), 
    conv2(16,3), 
    Flatten()
)

In [20]:
help(res_block(16))

Help on SequentialEx in module fastai.layers object:

class SequentialEx(fastai.torch_core.Module)
 |  SequentialEx(*layers)
 |  
 |  Like `nn.Sequential`, but with ModuleList semantics, and can access module input
 |  
 |  Method resolution order:
 |      SequentialEx
 |      fastai.torch_core.Module
 |      torch.nn.modules.module.Module
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, i)
 |  
 |  __init__(self, *layers)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  append(self, l)
 |  
 |  extend(self, l)
 |  
 |  forward(self, x)
 |      Defines the computation performed at every call.
 |      
 |      Should be overridden by all subclasses.
 |      
 |      .. note::
 |          Although the recipe for forward pass needs to be defined within
 |          this function, one should call the :class:`Module` instance afterwards
 |          instead of this since the former takes care of running the
 |          registered

In [21]:
def conv_and_res(ni,nf): 
    return nn.Sequential(conv2(ni,nf), ResBlock(nf))

In [22]:
model = nn.Sequential(
    conv_and_res(3,8),
    conv_and_res(8,16), 
    conv_and_res(16,32),
    conv_and_res(32,16),
    conv_and_res(16,3),
    Flatten()
)

In [23]:
learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

In [24]:
data

ImageDataBunch;

Train: LabelList (63 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
blink,blink,blink,blink,blink
Path: /home/sathiesh/ml/audio_classification/nbs/../data;

Valid: LabelList (27 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
blink,blink,blink,blink,blink
Path: /home/sathiesh/ml/audio_classification/nbs/../data;

Test: None

In [25]:
#learn.lr_find()
#learn.recorder.plot()

In [28]:
learn

Learner(data=ImageDataBunch;

Train: LabelList (63 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
blink,blink,blink,blink,blink
Path: /home/sathiesh/ml/audio_classification/nbs/../data;

Valid: LabelList (27 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: CategoryList
blink,blink,blink,blink,blink
Path: /home/sathiesh/ml/audio_classification/nbs/../data;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      (conv1): Sequential(
        (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ReLU(inplace)
        (2): BatchNorm2d(8, eps=1e-05,

In [27]:
learn.fit_one_cycle(1, slice(1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,4.678438,3.690741,0.000000,00:01


torch.Size([16, 8, 64, 64])
torch.Size([16, 8, 64, 64])
torch.Size([16, 8, 64, 64])
neste
torch.Size([16, 16, 32, 32])
torch.Size([16, 16, 32, 32])
torch.Size([16, 16, 32, 32])
neste
torch.Size([16, 32, 16, 16])
torch.Size([16, 32, 16, 16])
torch.Size([16, 32, 16, 16])
neste
torch.Size([16, 16, 8, 8])
torch.Size([16, 16, 8, 8])
torch.Size([16, 16, 8, 8])
neste
torch.Size([16, 3, 4, 4])
torch.Size([16, 3, 4, 4])
torch.Size([16, 3, 4, 4])
neste
torch.Size([16, 8, 64, 64])
torch.Size([16, 8, 64, 64])
torch.Size([16, 8, 64, 64])
neste
torch.Size([16, 16, 32, 32])
torch.Size([16, 16, 32, 32])
torch.Size([16, 16, 32, 32])
neste
torch.Size([16, 32, 16, 16])
torch.Size([16, 32, 16, 16])
torch.Size([16, 32, 16, 16])
neste
torch.Size([16, 16, 8, 8])
torch.Size([16, 16, 8, 8])
torch.Size([16, 16, 8, 8])
neste
torch.Size([16, 3, 4, 4])
torch.Size([16, 3, 4, 4])
torch.Size([16, 3, 4, 4])
neste
torch.Size([16, 8, 64, 64])
torch.Size([16, 8, 64, 64])
torch.Size([16, 8, 64, 64])
neste
torch.Size([16, 